# Transformer

Authors:
- Marina Debogović
- Marko Njegomir

## VIT transformer implemented in pytorch that is used as a baseline for comparison

[Code for regular implementation of VIT transformer in pytorch](https://towardsdatascience.com/a-demonstration-of-using-vision-transformers-in-pytorch-mnist-handwritten-digit-recognition-407eafbc15b0)

In [1]:
!pip install einops
!pip install wandb

In [2]:
import torch
from torch import nn
import time
from torch import optim
import wandb
from sklearn import metrics 
import os
import numpy as np

# torch.cuda.get_device_name(0)

In [3]:
import torch
import torch.nn.functional as F

from torch import nn
from einops import rearrange

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads=8):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5

        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
        self.to_out = nn.Linear(dim, dim)

    def forward(self, x, mask = None):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x)
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv=3, h=h)

        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale
        # matricno mnozenje q i k

        if mask is not None:
            mask = F.pad(mask.flatten(1), (1, 0), value = True)
            assert mask.shape[-1] == dots.shape[-1], 'mask has incorrect dimensions'
            mask = mask[:, None, :] * mask[:, :, None]
            dots.masked_fill_(~mask, float('-inf'))
            del mask

        attn = dots.softmax(dim=-1)

        out = torch.einsum('bhij,bhjd->bhid', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out =  self.to_out(out)
        return out

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Residual(PreNorm(dim, Attention(dim, heads = heads))),
                Residual(PreNorm(dim, FeedForward(dim, mlp_dim)))
            ]))

    def forward(self, x, mask=None):
        for attn, ff in self.layers:
            x = attn(x, mask=mask)
            x = ff(x)
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.transformer = Transformer(dim, depth, heads, mlp_dim)

        self.to_cls_token = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, img, mask=None):
        if mask: mask.to(device)
        p = self.patch_size

        x = rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = p, p2 = p)       
        x = x.to(device)
        x = self.patch_to_embedding(x)

        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        cls_tokens = cls_tokens.to(device)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.transformer(x, mask)

        x = self.to_cls_token(x[:, 0])
        x = x.to(device)
        return self.mlp_head(x)

In [4]:
import torch
import torchvision

# torch.manual_seed(42)
projectName = 'CIFAR10'
wandb.init(entity = 'njmarko', project = projectName)

DOWNLOAD_PATH = '/data/cifar10'
BATCH_SIZE_TRAIN = 250
BATCH_SIZE_VAL = 250
BATCH_SIZE_TEST = 250

mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]

transform_cifar10 = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                  torchvision.transforms.transforms.Normalize(mean, std)])
train_data = torchvision.datasets.CIFAR10(DOWNLOAD_PATH, train=True, download=True, transform=transform_cifar10)
train_set, val_set = torch.utils.data.random_split(train_data, [45000, 5000])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE_VAL, shuffle=True)
test_set = torchvision.datasets.CIFAR10(DOWNLOAD_PATH, train=False, download=True, transform=transform_cifar10)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE_TEST, shuffle=True)

wandb: Currently logged in as: njmarko. Use `wandb login --relogin` to force relogin


Files already downloaded and verified
Files already downloaded and verified


In [5]:
def train_epoch(model, optimizer, data_loader, loss_history, scheduler):
    total_samples = len(data_loader.dataset)
    model.train()

    correct_samples = 0
    for i, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = F.log_softmax(model(data), dim=1)

        target = target.to(device)
        output = output.to(device)
        
        _, pred = torch.max(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step()

        correct_samples += pred.eq(target).sum()
        target = target.cpu().detach().numpy()
        pred = pred.cpu().detach().numpy()
      
        
        f1_score = metrics.f1_score(target, pred, average='micro')

        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_history.append(loss.item())
            wandb.log({
                'train_loss': loss.item(),
                'train_f1_score': f1_score
            })
            
    acc = 100.0 * correct_samples / total_samples
    wandb.log({
        'train_accuracy': acc
    })
    print(f'Accuracy: ' + '{:4.2f}'.format(acc) + '%')        

In [6]:
def validate(model, data_loader, loss_history):
    model.eval()
    
    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0
    
    global_target = np.array([])
    global_pred = np.array([]) 

    with torch.no_grad():
        for data, target in data_loader:
            res = model(data)
            res = res.to(device)
            output = F.log_softmax(res, dim=1)
            target = target.to(device)
            output = output.to(device)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()
            
            target = target.cpu().detach().numpy()
            pred = pred.cpu().detach().numpy()
            
            global_target = np.concatenate((global_target, target))
            global_pred = np.concatenate((global_pred, pred))
            
    avg_loss = total_loss / total_samples
    acc = 100.0 * correct_samples / total_samples
    loss_history.append(avg_loss)

   
    f1_score = metrics.f1_score(global_target, global_pred, average='micro')

    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')
    
    wandb.log({
        'val_loss': loss.item(),
        'val_f1_score': f1_score,
        'val_accuracy': acc
    })
    return acc

In [7]:
def evaluate(model, data_loader, loss_history):
    model.eval()
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0
    
    global_target = np.array([])
    global_pred = np.array([]) 

    with torch.no_grad():
        for data, target in data_loader:
            res = model(data)
            res = res.to(device)
            output = F.log_softmax(res, dim=1)
            target = target.to(device)
            output = output.to(device)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()
            
            target = target.cpu().detach().numpy()
            pred = pred.cpu().detach().numpy()
            
            global_target = np.concatenate((global_target, target))
            global_pred = np.concatenate((global_pred, pred))

    avg_loss = total_loss / total_samples
    acc = 100.0 * correct_samples / total_samples
    loss_history.append(avg_loss)

    f1_score = metrics.f1_score(global_target, global_pred, average='micro')
    precision = metrics.precision_score(global_target, global_pred, average='micro')
    recall = metrics.recall_score(global_target, global_pred, average='micro')

#     wandb.log({
#         'test_loss': loss.item(),
#         'accuracy': acc,
#         'test_f1_score': f1_score,
#         'precision': precision,
#         'recall': recall
#     })
    print('\nTest loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(acc) + '%)  Precision: ' + '{:4.2f}'.format(precision) +
          '  Recall: ' + '{:4.2f}'.format(recall) + '\n')
    
    cm = metrics.confusion_matrix(global_target, global_pred)
    print(f'Confusion matrix:\n {cm}')

In [8]:
if not os.path.exists('CIFAR10-classic'): os.makedirs('CIFAR10-classic')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

N_EPOCHS = 20
best_acc = -1
best_epoch = -1

path = 'CIFAR10-classic'
if not os.path.exists(path): os.makedirs(path)

start_time = time.time()
model = ViT(image_size=32, patch_size=8, num_classes=10, channels=3,
            dim=128, depth=8, heads=8, mlp_dim=256)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.05)
early_stop_tolerance = 10e-4
model = model.to(device)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.001, cycle_momentum=False, step_size_up = 1000)


train_loss_history, val_loss_history, test_loss_history = [], [], []
for epoch in range(1, N_EPOCHS + 1):
    early_stopping = 0
    print('Epoch:', epoch)
    train_epoch(model, optimizer, train_loader, train_loss_history, scheduler)
    acc = validate(model, val_loader, val_loss_history)
        
    if best_acc < acc:
        best_acc = acc
        torch.save(model.state_dict(), os.path.join(path, f'classic_transfromer-{projectName}-acc{acc}'))

    if len(train_loss_history) > 2 and np.isclose(train_loss_history[-2], train_loss_history[-1], atol=early_stop_tolerance):
        early_stopping += 1
        if (early_stopping == 5):
            print(f"Early stop with tolerance {early_stop_tolerance} for losses {train_loss_history[-2]} and {train_loss_history[-1]}")
            break
    else:
        early_stopping = 0
        
# Testiranje modela
path2 = f'CIFAR10-classic/classic_transfromer-{projectName}-acc{best_acc}'
print(path2)
model.load_state_dict(torch.load(path2))
model = model.to(device)
evaluate(model, test_loader, test_loss_history)

        
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/45000 (  0%)]  Loss: 2.3568
[25000/45000 ( 56%)]  Loss: 1.8127
Accuracy: 28.47%

Average test loss: 1.7078  Accuracy: 1885/ 5000 (37.70%)


Test loss: 1.6896  Accuracy: 3881/10000 (38.81%)  Precision: 0.39  Recall: 0.39

Confusion matrix:
 [[542  64  79  14   4   4  21  27 182  63]
 [ 63 500  14  26   8   7  53  36  97 196]
 [146  42 404  65  91  35  97  68  35  17]
 [ 80  63 154 275  46  99 106  88  34  55]
 [ 61  36 336  49 213  30 155  75  27  18]
 [ 55  44 162 211  48 211  87 102  55  25]
 [ 33  27 250  73 117  34 360  64  14  28]
 [ 77  55 104  74  94  56  55 380  29  76]
 [184 104  25  24   6  21   7  19 531  79]
 [ 77 182  11  30   5   8  28  68 126 465]]
Execution time: 30.19 seconds
